In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [ ]:
class StockFeedforwardNN(nn.Module):
    def __init__(self, 
                 vocab_size,  # news article titles
                 news_embed_dim=128,
                 stock_vocab_size=400,  # stock name, might be too big?
                 stock_embed_dim=16,
                 num_numeric_features=20, # for numeric features, open, close, etc
                 hidden_size=256):
        super(StockFeedforwardNN, self).__init__()

        self.news_embed = nn.EmbeddingBag(vocab_size, news_embed_dim, mode='mean')  # mean pooling
        self.stock_embed = nn.Embedding(stock_vocab_size, stock_embed_dim)

        total_input_size = news_embed_dim + stock_embed_dim + num_numeric_features

        self.net = nn.Sequential(
            nn.Linear(total_input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.BatchNorm1d(hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size // 2, 1)
        )

    def forward(self, news_input, news_offsets, stock_id, numeric_feats):
        news_vec = self.news_embed(news_input, news_offsets)
        stock_vec = self.stock_embed(stock_id)
        x = torch.cat([news_vec, stock_vec, numeric_feats], dim=1)
        return self.net(x)